In [1]:
import pandas as pd
import json

In [2]:
with open('../data/data_salto/concursos/concursos_salto_nacional_2025.json', 'r', encoding='utf-8') as file:
    datos_concursos = json.load(file)

df_concursos = pd.DataFrame(datos_concursos)
df_concursos = df_concursos.rename(columns = {"Nombre": "Concurso", "Categoría": "Categoria_concurso"})
df_concursos["Concurso"] = df_concursos["Concurso"].str.strip()
df_concursos.head()

,Concurso,Categoria_concurso,Provincia,Localidad,Disciplina,Federación,Resultados,País,Inicio,Final,Ambito
0,CSN3* Invierno,CSN***,Sevilla,Real Club Pineda,Salto de Obstáculos,Federación Hípica Andaluza,Ver,Epaña,10/01/2025,12/01/2025,Nacional
1,SPRING MET I 2025 YH*,CSI YH*,Valencia,Centro Ecuestre Oliva Nova,Salto de Obstáculos,Federación Hípica Comunidad Valenciana,Ver,Epaña,14/01/2025,16/01/2025,Internacional
2,SPRING MET I 2025 - SILVER 1*,CSI*,Valencia,Centro Ecuestre Oliva Nova,Salto de Obstáculos,Federación Hípica Comunidad Valenciana,Ver,Epaña,16/01/2025,18/01/2025,Internacional
3,SPRING MET I 2025 - GOLD 2*,CSI**,Valencia,Centro Ecuestre Oliva Nova,Salto de Obstáculos,Federación Hípica Comunidad Valenciana,Ver,Epaña,16/01/2025,19/01/2025,Internacional
4,CSIYH* - 1st Week Andalucía Pre-Sunshine Tour ...,CSI YH*,Cádiz,Dehesa Montenmedio,Salto de Obstáculos,Federación Hípica Andaluza,Ver,Epaña,16/01/2025,19/01/2025,Internacional


In [3]:
df_pruebas_resultados = pd.read_csv("../data/data_salto/resultados_combinados_2025.csv", index_col=0)
df_pruebas_resultados.head()

C:\Users\gabri\AppData\Local\Temp\ipykernel_18376\1077302799.py:1: DtypeWarning: Columns (3,6) have mixed types. Specify dtype option on import or set low_memory=False.
  df_pruebas_resultados = pd.read_csv("../data/data_salto/resultados_combinados_2025.csv", index_col=0)


,Estado,Posicion,Lic_jinete,Jinete,Pais_jinete,Lic_caballo,Caballo,Raza_caballo,Pais_caballo,Puntuacion,...,Categoria,Ptos_obs_R1,Ptos_tiempo_R1,Tiempo_R1,Ptos_obs_R2,Ptos_tiempo_R2,Tiempo_R2,Ptos_obs_R3,Ptos_tiempo_R3,Tiempo_R3
0,FIN,1,284725,Pedro Miguel DA SILVA MANSO,ESP,169882,BRIDGETOWN JONES OS,(Indt.),ESP,"0/59,91",...,CSN***,0.0,NaN,59.91,NaN,NaN,NaN,NaN,NaN,NaN
1,FIN,2,77612,Ignacio ASTOLFI LÓPEZ,ESP,157373,QUINTO BLUE PS,OLDBG,ESP,"0/63,23",...,CSN***,0.0,NaN,63.23,NaN,NaN,NaN,NaN,NaN,NaN
2,FIN,3,306183,Francisco PICAO CALDEIRA,ESP,145996,FLOR DU PASO,CRPOR,ESP,"0/65,58",...,CSN***,0.0,NaN,65.58,NaN,NaN,NaN,NaN,NaN,NaN
3,FIN,4,1004,Jesús TORRES GARCÍA,ESP,170075,DIABETTO PS,OLDBG,ESP,"0/66,00",...,CSN***,0.0,NaN,66.00,NaN,NaN,NaN,NaN,NaN,NaN
4,FIN,5,367363,Nina FAGERSTROM,ESP,170155,HASENACHER CERVIN,ZANG,ESP,"0/66,78",...,CSN***,0.0,NaN,66.78,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
df_final = pd.merge(df_pruebas_resultados, df_concursos, on = "Concurso", how = "inner")
df_final['Fecha_prueba'] = pd.to_datetime(df_final['Fecha_prueba'])
df_final['Inicio'] = pd.to_datetime(df_final['Inicio'], dayfirst=True)
df_final['Final'] = pd.to_datetime(df_final['Final'], dayfirst=True)
df_final = df_final[(df_final['Fecha_prueba'] >= df_final['Inicio']) & (df_final['Fecha_prueba'] <= df_final['Final'])]
df_final = df_final.drop(['Puntuacion', 'Resultados', 'Categoria_concurso', 'Pais_jinete', 'Pais_caballo', 'Raza_caballo'], axis=1)
df_final["Jinete"] = df_final["Jinete"].str.lower()
df_final["Caballo"] = df_final["Caballo"].str.lower()
df_final["Lic_jinete"] = df_final["Lic_jinete"].astype(str)
df_final["Lic_caballo"] = df_final["Lic_caballo"].astype(str)
# df_final["Raza_caballo"] = df_final["Raza_caballo"].apply(lambda x: None if x == '(Indt.)' else x)
df_final["Lic_jinete"] = df_final["Lic_jinete"].apply(lambda x: "10186822" if x == '--' else x)
df_final["Jinete"] = df_final["Jinete"].apply(lambda x: "jesus folch redondo" if x == '-- -- --' else x)
lista = ["ELI", "RET", "NOP"]
df_final["Posicion"] = df_final["Posicion"].apply(lambda x: None if x in lista else x)
df_final["Lic_caballo"] = df_final["Lic_caballo"].apply(lambda x: None if x == '--' else x)

lic_mas_larga_por_jinete = (
    df_final[["Jinete", "Lic_jinete"]]
    .drop_duplicates()
    .assign(longitud=lambda x: x["Lic_jinete"].astype(str).str.len())
    .sort_values("longitud", ascending=False)
    .drop_duplicates(subset="Jinete")
    .drop(columns="longitud")
    .set_index("Jinete")
)

lic_mas_larga_por_caballo = (
    df_final[["Caballo", "Lic_caballo"]]
    .drop_duplicates()
    .assign(longitud=lambda x: x["Lic_caballo"].astype(str).str.len())
    .sort_values("longitud", ascending=False)
    .drop_duplicates(subset="Caballo")
    .drop(columns="longitud")
    .set_index("Caballo")
)

# 2. Reemplazar en el DataFrame original
df_final["Lic_jinete"] = df_final["Jinete"].map(lic_mas_larga_por_jinete["Lic_jinete"])
df_final["Lic_caballo"] = df_final["Caballo"].map(lic_mas_larga_por_caballo["Lic_caballo"])
df_final.head()

,Estado,Posicion,Lic_jinete,Jinete,Lic_caballo,Caballo,Premio,Dinero_premio,Prueba,Fecha_prueba,...,Ptos_tiempo_R3,Tiempo_R3,Provincia,Localidad,Disciplina,Federación,País,Inicio,Final,Ambito
0,FIN,1,284725,pedro miguel da silva manso,169882,bridgetown jones os,True,450.0,"1,30 - A c.c. (238.2.1)",2025-01-10,...,NaN,NaN,Sevilla,Real Club Pineda,Salto de Obstáculos,Federación Hípica Andaluza,Epaña,2025-01-10,2025-01-12,Nacional
1,FIN,2,77612,ignacio astolfi lópez,157373,quinto blue ps,True,300.0,"1,30 - A c.c. (238.2.1)",2025-01-10,...,NaN,NaN,Sevilla,Real Club Pineda,Salto de Obstáculos,Federación Hípica Andaluza,Epaña,2025-01-10,2025-01-12,Nacional
2,FIN,3,306183,francisco picao caldeira,145996,flor du paso,True,200.0,"1,30 - A c.c. (238.2.1)",2025-01-10,...,NaN,NaN,Sevilla,Real Club Pineda,Salto de Obstáculos,Federación Hípica Andaluza,Epaña,2025-01-10,2025-01-12,Nacional
3,FIN,4,10006303,jesús torres garcía,107ZP87,diabetto ps,True,150.0,"1,30 - A c.c. (238.2.1)",2025-01-10,...,NaN,NaN,Sevilla,Real Club Pineda,Salto de Obstáculos,Federación Hípica Andaluza,Epaña,2025-01-10,2025-01-12,Nacional
4,FIN,5,10002400,nina fagerstrom,105ZA09,hasenacher cervin,True,125.0,"1,30 - A c.c. (238.2.1)",2025-01-10,...,NaN,NaN,Sevilla,Real Club Pineda,Salto de Obstáculos,Federación Hípica Andaluza,Epaña,2025-01-10,2025-01-12,Nacional


In [5]:
df_final.to_csv("../data/data_salto/informacion_concursos_final.csv")

In [6]:
df = pd.read_csv("../data/data_salto/informacion_concursos_final.csv", index_col = 0)
df["Disciplina"] = "Salto de Obstaculos"
i = 0
df["id_resultado"] = range(i +1 , i + 1 + len(df))
print(df["Lic_jinete"].duplicated().sum())
print(df["Jinete"].duplicated().sum())

69458
69458


C:\Users\gabri\AppData\Local\Temp\ipykernel_18376\1380352046.py:1: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("../data/data_salto/informacion_concursos_final.csv", index_col = 0)
